In [1]:
#Importing Libraries

import numpy as np
import cvxpy as cvx
import pandas as pd
from datetime import datetime
import pandas_datareader.data as pdr

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


## Optimization with cvxpy

http://www.cvxpy.org/

Finding the optimal weights on a m-asset portfolio given the covariance matrix of the returns of m-stocks .And creating a function that takes in these values as arguments and returns the vector of optimal weights.


### optimization
So now that we have our objective function and constraints, we can solve for the values of $\mathbf{x}$.
cvxpy has the constructor `Problem(objective, constraints)`, which returns a `Problem` object.

The `Problem` object has a function solve(), which returns the minimum of the solution.  In this case, this is the minimum variance of the portfolio.

It also updates the vector $\mathbf{x}$.

We can check out the values of vector of weights x that gave the minimum portfolio variance by using `x.value`

In [0]:
def optimize_portfolio(returns):
    
    
    m = len(returns)
    #covariance matrix of returns
    cov = np.cov(returns)
    
    # x variables (to be found with optimization)
    x = cvx.Variable(m)
    
    #portfolio variance, in quadratic form
    portfolio_variance = cvx.quad_form(x,cov)    
    
    #objective function
    objective = cvx.Minimize(portfolio_variance)
    
    #constraints
    constraints = [x >= 0, sum(x) == 1]

    #use cvxpy to solve the objective
    prob = cvx.Problem(objective, constraints)
    prob.solve()
    x_values = x.value

    return x_values

In [0]:
#Stock_Data_Extraction

tickers = ["ASIANPAINT.NS","ADANIPORTS.NS","AXISBANK.NS","BAJAJ-AUTO.NS"]

start= datetime(2018,1,1)
end = datetime.today()

#Creeating a dictionary to store data for all stocks.
ohlc_daily = {}
for i in range(len(tickers)):
    ohlc_daily[tickers[i]] = pdr.get_data_yahoo(tickers[i],start = start , end = end)

#Creating a dataframe to store daily_returns of all stocks
returns = pd.DataFrame()
for ticker in tickers:
    returns[ticker] = (ohlc_daily[ticker]['Adj Close'].pct_change())
returns = returns.dropna(axis=0)

#Applying Optimization function to get min_variance_weights
min_var_weights = optimize_portfolio(returns.T)
# print(min_var_weights)


In [18]:
for i in range(len(min_var_weights)):
  print(tickers[i],"is =====",min_var_weights[i]);

ASIANPAINT.NS is ===== 0.4688522709880375
ADANIPORTS.NS is ===== 0.16140489653268553
AXISBANK.NS is ===== -7.066324258965074e-20
BAJAJ-AUTO.NS is ===== 0.3697428324792769
